In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import math

# Parameters
learning_rate = 0.0001 #0.001
display_step = 1

C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\___Anaconda\envs\forTFgpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

In [2]:
import csv
import os
import numpy as np

#인코딩중요
#라벨만들기
with open('./trainSet.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_Y = [y[1] for y in data]

_Y = np.array(_Y)
_Y = _Y.astype(float) # shape(-1)
_Y = [[1-y, y] for y in _Y] # shape(-1, 2)



#데이터만들기
with open('./features4096.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
#이름빼기 
_X1 = [x[1:] for x in data]
_X1 = np.array(_X1)
_X1 = _X1.astype(float) # shape(-1, 1024)


#데이터만들기
with open('./features_3g_16.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
#이름빼기
_X2 = [x[1:] for x in data]
_X2 = np.array(_X2)
_X2 = _X2.astype(float) # shape(-1, 1024)


#데이터만들기
with open('./features_4g_8.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
#이름빼기
_X3 = [x[1:] for x in data]
_X3 = np.array(_X3)
_X3 = _X3.astype(float) # shape(-1, 1024)


#데이터만들기
with open('./features_5g.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
#이름빼기
_X4 = [x[1:] for x in data]
_X4 = np.array(_X4)
_X4 = _X4.astype(float) # shape(-1, 1024)


#데이터만들기
with open('./features_6g.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])
#이름빼기
_X5 = [x[1:] for x in data]
_X5 = np.array(_X5)
_X5 = _X5.astype(float) # shape(-1, 1024)


all_set = list(zip(_X1,_X2,_X3,_X4,_X5, _Y)) # shape(10000, 6, ~)
#M_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][1] == 1] # 7000
#N_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][0] == 1] # 3000
#np.random.shuffle(M_all_set)
#np.random.shuffle(N_all_set)

# 악성,정상 개수 맞추기
#if len(M_all_set) > len(N_all_set):
#    M_all_set = M_all_set[:len(N_all_set)]
#else:
#    N_all_set = N_all_set[:len(M_all_set)]

#all_set = M_all_set + N_all_set




In [3]:
#tester
K = 0
train_set = [all_set[i] for i in range(len(all_set)) if (i-K)%5 != 0]
test_set = [all_set[i] for i in range(len(all_set)) if (i-K)%5 == 0]

#np.random.shuffle(all_set) # 6000

#train_set = all_set[len(all_set)//5:] # 4800
#test_set = all_set[:len(all_set)//5] # 1200

train_Xs = []
test_Xs = []
for i in range(5):
    train_X = np.array([len6[i] for len6 in train_set])
    train_Xs.append(train_X)
    test_X = np.array([len6[i] for len6 in test_set])
    test_Xs.append(test_X)
    
train_Y = np.array([len6[5] for len6 in train_set])
test_Y = np.array([len6[5] for len6 in test_set])
    
M_test_Xs = []
N_test_Xs = []
for i in range(5):
    M_test_X = np.array([len6[i] for len6 in test_set if len6[5][1] == 1])
    M_test_Xs.append(M_test_X)
    N_test_X = np.array([len6[i] for len6 in test_set if len6[5][0] == 1])
    N_test_Xs.append(N_test_X)
    
M_test_Y = np.array([len6[5] for len6 in test_set if len6[5][1] == 1])
N_test_Y = np.array([len6[5] for len6 in test_set if len6[5][0] == 1])
    
#M_test_X1 = [X1 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
#M_test_X2 = [X2 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
#M_test_X3 = [X3 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
#M_test_X4 = [X4 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
#M_test_X5 = [X5 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
#M_test_Y = [Y for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면

#N_test_X1 = [X1 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
#N_test_X2 = [X2 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
#N_test_X3 = [X3 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
#N_test_X4 = [X4 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
#N_test_X5 = [X5 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
#N_test_Y = [Y for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면


In [4]:
#tf.reset_default_graph()

with tf.device('/gpu:0'):
    Xs = []
    for i in range(5):
        X = tf.placeholder(tf.float32, [None, 4096])
        Xs.append(X)
    Y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)
    
    
    W = {}
    b = {}
    L = {}
    for i in range(6):
        W[i] = {}
        b[i] = {}
        L[i] = {}
    
    hypothesis = [None] * 5
    cost = {}
    optimizer = {}

    model = [4096,2000,1000,500,250,100,2]
    for i in range(5):
        for j in range(6):
            W[j][i] = tf.get_variable('w%d_%d'%(j,i), shape=[model[j],model[j+1]], initializer=tf.contrib.layers.xavier_initializer())
            b[j][i] = tf.Variable(tf.random_normal([model[j+1]]), name='w%d_%d'%(j,i))
        L[0][i] = tf.nn.relu(tf.matmul(Xs[i], W[0][i]) + b[0][i])
        L[0][i] = tf.nn.dropout(L[0][i], keep_prob=keep_prob)
        for j in range(4):
            L[j+1][i] = tf.nn.relu(tf.matmul(L[j][i], W[j+1][i]) + b[j+1][i])
            L[j+1][i] = tf.nn.dropout(L[j+1][i], keep_prob=keep_prob)
        hypothesis[i] = tf.matmul(L[4][i], W[5][i]) + b[5][i]
        
        cost[i] = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis[i], labels=Y))
        optimizer[i] = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost[i])
   
'''
    for i in range(5):
        # Layer1
        W[0][i] = tf.get_variable('%f'%np.random.random(), shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())
        b[0][i] = tf.Variable(tf.random_normal([2000]), name='%f'%np.random.random())
        L[0][i] = tf.nn.relu(tf.matmul(Xs[i], W[0][i]) + b[0][i])
        L[0][i] = tf.nn.dropout(L[0][i], keep_prob=keep_prob)
        # Layer2
        W[1][i] = tf.get_variable('%f'%np.random.random(), shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())
        b[1][i] = tf.Variable(tf.random_normal([1000]), name='%f'%np.random.random())
        L[1][i] = tf.nn.relu(tf.matmul(L[0][i], W[1][i]) + b[1][i])
        L[1][i] = tf.nn.dropout(L[1][i], keep_prob=keep_prob)
        # Layer3
        W[2][i] = tf.get_variable('%f'%np.random.random(), shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())
        b[2][i] = tf.Variable(tf.random_normal([500]), name='%f'%np.random.random())
        L[2][i] = tf.nn.relu(tf.matmul(L[1][i], W[2][i]) + b[2][i])
        L[2][i] = tf.nn.dropout(L[2][i], keep_prob=keep_prob)
        # Layer4
        W[3][i] = tf.get_variable('%f'%np.random.random(), shape=[500,250], initializer=tf.contrib.layers.xavier_initializer())
        b[3][i] = tf.Variable(tf.random_normal([250]), name='%f'%np.random.random())
        L[3][i] = tf.nn.relu(tf.matmul(L[2][i], W[3][i]) + b[3][i])
        L[3][i] = tf.nn.dropout(L[3][i], keep_prob=keep_prob)
        # Layer5
        W[4][i] = tf.get_variable('%f'%np.random.random(), shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
        b[4][i] = tf.Variable(tf.random_normal([100]), name='%f'%np.random.random())
        L[4][i] = tf.nn.relu(tf.matmul(L[3][i], W[4][i]) + b[4][i])
        L[4][i] = tf.nn.dropout(L[4][i], keep_prob=keep_prob)
        # Layer6
        W[5][i] = tf.get_variable('%f'%np.random.random(), shape=[100,2], initializer=tf.contrib.layers.xavier_initializer())
        b[5][i] = tf.Variable(tf.random_normal([2]), name='%f'%np.random.random())
        hypothesis[i] = tf.matmul(L[4][i], W[5][i]) + b[5][i]
        
        cost[i] = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis[i], labels=Y))
        optimizer[i] = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost[i])
'''

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



"\n    for i in range(5):\n        # Layer1\n        W[0][i] = tf.get_variable('%f'%np.random.random(), shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())\n        b[0][i] = tf.Variable(tf.random_normal([2000]), name='%f'%np.random.random())\n        L[0][i] = tf.nn.relu(tf.matmul(Xs[i], W[0][i]) + b[0][i])\n        L[0][i] = tf.nn.dropout(L[0][i], keep_prob=keep_prob)\n        # Layer2\n        W[1][i] = tf.get_variable('%f'%np.random.random(), shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())\n        b[1][i] = tf.Variable(tf.random_normal([1000]), name='%f'%np.random.random())\n        L[1][i] = tf.nn.relu(tf.matmul(L[0][i], W[1][i]) + b[1][i])\n        L[1][i] = tf.nn.dropout(L[1][i], keep_prob=keep_prob)\n        # Layer3\n        W[2][i] = tf.get_variable('%f'%np.random.random(), shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())\n        b[2][i] = tf.Variable(tf.random_normal([500]), name='%f'%np.random.random())\n     

In [ ]:
 '''
 for i in range(5):
        # Layer1
        W[0][i] = tf.get_variable('w1_%d'%i, shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())
        b[0][i] = tf.Variable(tf.random_normal([2000]), name='b1_%d'%i)
        L[0][i] = tf.nn.relu(tf.matmul(Xs[i], W[0][i]) + b[0][i])
        L[0][i] = tf.nn.dropout(L[0][i], keep_prob=keep_prob)
        # Layer2
        W[1][i] = tf.get_variable('w2_%d'%i, shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())
        b[1][i] = tf.Variable(tf.random_normal([1000]), name='b2_%d'%i)
        L[1][i] = tf.nn.relu(tf.matmul(L[0][i], W[1][i]) + b[1][i])
        L[1][i] = tf.nn.dropout(L[1][i], keep_prob=keep_prob)
        # Layer3
        W[2][i] = tf.get_variable('w3_%d'%i, shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())
        b[2][i] = tf.Variable(tf.random_normal([500]), name='b3_%d'%i)
        L[2][i] = tf.nn.relu(tf.matmul(L[1][i], W[2][i]) + b[2][i])
        L[2][i] = tf.nn.dropout(L[2][i], keep_prob=keep_prob)
        # Layer4
        W[3][i] = tf.get_variable('w4_%d'%i, shape=[500,250], initializer=tf.contrib.layers.xavier_initializer())
        b[3][i] = tf.Variable(tf.random_normal([250]), name='b4_%d'%i)
        L[3][i] = tf.nn.relu(tf.matmul(L[2][i], W[3][i]) + b[3][i])
        L[3][i] = tf.nn.dropout(L[3][i], keep_prob=keep_prob)
        # Layer5
        W[4][i] = tf.get_variable('w5_%d'%i, shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
        b[4][i] = tf.Variable(tf.random_normal([100]), name='b5_%d'%i)
        L[4][i] = tf.nn.relu(tf.matmul(L[3][i], W[4][i]) + b[4][i])
        L[4][i] = tf.nn.dropout(L[4][i], keep_prob=keep_prob)
        # Layer6
        W[5][i] = tf.get_variable('w6_%d'%i, shape=[100,2], initializer=tf.contrib.layers.xavier_initializer())
        b[5][i] = tf.Variable(tf.random_normal([2]), name='b6_%d'%i)
        hypothesis[i] = tf.matmul(L[4][i], W[5][i]) + b[5][i]
        
        cost[i] = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis[i], labels=Y))
        optimizer[i] = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost[i])
'''

In [5]:
param_list = []
for i in range(6):
    for j in range(5):
        param_list.append(W[i][j])
        param_list.append(b[i][j])
        
    
saver = tf.train.Saver(param_list)

In [14]:
training_epochs = 10 #15
batch_size = 30
new_or_load = 1


# Training
sess = tf.Session()
if new_or_load == 0:
    sess.run(tf.global_variables_initializer())
else:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, './tf_model_test/tf_model.ckpt')


INFO:tensorflow:Restoring parameters from ./tf_model_test/tf_model.ckpt


In [15]:

avg_costs = []
# Training cycle
for epoch in range(training_epochs):
    for i in range(5):
        avg_cost = 0
        avg_costs.append(avg_cost)
    total_batch = math.ceil(len(train_Y) / batch_size)

    for i in range(total_batch):
        batch_xss = []
        for j in range(5):
            batch_xs = train_Xs[j][i*batch_size:(i+1)*batch_size]
            batch_xss.append(batch_xs)
        batch_ys = train_Y[i*batch_size:(i+1)*batch_size]
        
        
        for j in range(5):
            feed_dict = {Xs[j] : batch_xss[j], Y : batch_ys, keep_prob : 0.7}
            c, _ = sess.run([cost[j], optimizer[j]], feed_dict=feed_dict)
            avg_costs[j] += c * len(batch_ys) #뭉치갯수로 곱하기
        #time.sleep(0.001)
    
    #saver.save(sess, './tf_model/tf_model.ckpt')
    for i in range(5):
        avg_costs[i] /= len(train_Y) #전체개수로 나누기
    
    if (epoch+1)%5 == 0:
        print('Epoch: %03d'%(epoch+1), end='')
        for i in range(5):
            print('\tcost%d='%i, '{:.4f}'.format(avg_costs[i]), end='')
        print()
        
print(len(train_Y),' 개 학습완료')

Epoch: 005	cost0= 0.1968	cost1= 0.1782	cost2= 0.1724	cost3= 0.1781	cost4= 0.1985
Epoch: 010	cost0= 0.1634	cost1= 0.1508	cost2= 0.1447	cost3= 0.1475	cost4= 0.1708
8000  개 학습완료


In [16]:
# TEST

#is_correct = tf.equal(tf.math.argmax(tf.round(tf.nn.softmax(hypothesis_1)) + \
#                                 tf.round(tf.nn.softmax(hypothesis_2)) + \
#                                 tf.round(tf.nn.softmax(hypothesis_3)) + \
#                                 tf.round(tf.nn.softmax(hypothesis_4)) + \
#                                 tf.round(tf.nn.softmax(hypothesis_5)), 1), tf.arg_max(Y, 1))

#is_correct = tf.equal(tf.math.argmax(tf.nn.softmax(hypothesis_1) + \
#                                 tf.nn.softmax(hypothesis_2) + \
#                                 tf.nn.softmax(hypothesis_3) + \
#                                 tf.nn.softmax(hypothesis_4) + \
#                                 tf.nn.softmax(hypothesis_5), 1), tf.arg_max(Y, 1))
#print(hypothesis)

is_correct = tf.equal(tf.math.argmax(tf.reduce_sum(hypothesis, axis=0), 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

#feed_dict = {X: test_X, Y: test_Y, keep_prob: 1}
#print("Accuracy : ", accuracy.eval(session=sess, feed_dict=feed_dict))

#feed_dict = {X: M_test_X, Y: M_test_Y, keep_prob: 1}
#print("정탐율 : ", accuracy.eval(session=sess, feed_dict=feed_dict))

#feed_dict = {X: N_test_X, Y: N_test_Y, keep_prob: 1}
#print("오탐율 : ", 1 - accuracy.eval(session=sess, feed_dict=feed_dict))

total_batch = math.ceil(len(test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xss = []
    for j in range(5):
        batch_xs = test_Xs[j][i*batch_size:(i+1)*batch_size]
        batch_xss.append(batch_xs)
    batch_ys  = test_Y[i*batch_size:(i+1)*batch_size]
    
    feed_dict = {}
    for i in range(5):
        feed_dict[Xs[i]] = batch_xss[i]
    feed_dict[Y] = batch_ys
    feed_dict[keep_prob] = 1
    
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("Accuracy : %0.4f"%(total_acc/len(test_Y)))


total_batch = math.ceil(len(M_test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xss = []
    for j in range(5):
        batch_xs = M_test_Xs[j][i*batch_size:(i+1)*batch_size]
        batch_xss.append(batch_xs)
    batch_ys  = M_test_Y[i*batch_size:(i+1)*batch_size]
    
    feed_dict = {}
    for i in range(5):
        feed_dict[Xs[i]] = batch_xss[i]
    feed_dict[Y] = batch_ys
    feed_dict[keep_prob] = 1
    
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("정탐율 : %0.4f"%(total_acc/len(M_test_Y)))


total_batch = math.ceil(len(N_test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xss = []
    for j in range(5):
        batch_xs = N_test_Xs[j][i*batch_size:(i+1)*batch_size]
        batch_xss.append(batch_xs)
    batch_ys  = N_test_Y[i*batch_size:(i+1)*batch_size]
    
    feed_dict = {}
    for i in range(5):
        feed_dict[Xs[i]] = batch_xss[i]
    feed_dict[Y] = batch_ys
    feed_dict[keep_prob] = 1
    
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("오탐율 : %0.4f"%(1 - total_acc/len(N_test_Y)))



#r = rd.randint(0, len(test_X) - 1)
#print('Label:', sess.run(tf.argmax(test_Y[r:r+1], 1)))

#feed_dict = {X: test_X[r:r+1], keep_prob: 1}
#print('Prediction:', sess.run(tf.argmax(hypothesis, 1), feed_dict=feed_dict))

Instructions for updating:
Use `tf.math.argmax` instead
Accuracy : 0.9260
정탐율 : 0.9950
오탐율 : 0.2323


In [12]:
param_list = [W1_1,b1_1, W1_2,b1_2, W1_3,b1_3, W1_4,b1_4, W1_5,b1_5, \
                W2_1,b2_1, W2_2,b2_2, W2_3,b2_3, W2_4,b2_4, W2_5,b2_5, \
                W3_1,b3_1, W3_2,b3_2, W3_3,b3_3, W3_4,b3_4, W3_5,b3_5, \
                W4_1,b4_1, W4_2,b4_2, W4_3,b4_3, W4_4,b4_4, W4_5,b4_5, \
                W5_1,b5_1, W5_2,b5_2, W5_3,b5_3, W5_4,b5_4, W5_5,b5_5, \
                W6_1,b6_1, W6_2,b6_2, W6_3,b6_3, W6_4,b6_4, W6_5,b6_5]
    
saver = tf.train.Saver(param_list)
#saver.save(sess, './tf_model/tf_model.ckpt')
saver.save(sess, './tf_model_test/tf_model.ckpt')

'./tf_model/tf_model.ckpt'

In [11]:

#saver.save(sess, './tf_model/tf_model.ckpt')
saver.save(sess, './tf_model_test/tf_model.ckpt')

'./tf_model_test/tf_model.ckpt'

In [ ]:
#tester
K = 0
#train_set = [all_set[i] for i in range(len(all_set)) if (i-K)%5 != 0]
test_set = [all_set[i] for i in range(len(all_set)) if (i-K)%5 == 0]

test_X1 = np.array([X1 for X1,X2,X3,X4,X5,Y in test_set])
test_X2 = np.array([X2 for X1,X2,X3,X4,X5,Y in test_set])
test_X3 = np.array([X3 for X1,X2,X3,X4,X5,Y in test_set])
test_X4 = np.array([X4 for X1,X2,X3,X4,X5,Y in test_set])
test_X5 = np.array([X5 for X1,X2,X3,X4,X5,Y in test_set])
test_Y = np.array([Y for X1,X2,X3,X4,X5,Y in test_set])

M_test_X1 = [X1 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_X2 = [X2 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_X3 = [X3 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_X4 = [X4 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_X5 = [X5 for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_Y = [Y for X1,X2,X3,X4,X5,Y in test_set if Y[1] == 1] # label의 악성이면

N_test_X1 = [X1 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_X2 = [X2 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_X3 = [X3 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_X4 = [X4 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_X5 = [X5 for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_Y = [Y for X1,X2,X3,X4,X5,Y in test_set if Y[0] == 1] # label의 정상이면
